In [3]:
os.chdir('/content/drive/MyDrive/Data/Data/2023-retail')

In [2]:
%%time
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
import datetime
import os

CPU times: user 633 ms, sys: 65.2 ms, total: 698 ms
Wall time: 789 ms


In [7]:
import os

folder_path = "/content/drive/MyDrive/Data/Data/2024-retail"  # Change this to your folder path

for filename in os.listdir(folder_path):
    old_path = os.path.join(folder_path, filename)
    new_path = os.path.join(folder_path, os.path.splitext(filename)[0] + ".html")

    if os.path.isfile(old_path):  # Ensure it's a file, not a directory
        os.rename(old_path, new_path)

print("All files renamed to .html")


All files renamed to .html


In [5]:
import pandas as pd
import os

# Initialize an empty list to store rows
data_rows = []

# Loop through each file from 1.html to 31.html
for day in range(1, 12):
    file_name = f"1-{day}-24.html"

    if os.path.exists(file_name):
        try:
            # Read all tables in the HTML file
            df_list = pd.read_html(file_name)

            if df_list:
                df = df_list[0]  # Select the first table

                # Extract row names as index
                df = df.set_index(df.columns[0])  # First column is row names

                # Create a dictionary for the row
                formatted_row = {"Date": f"{day:02d}/2023"}

                for row_name in df.index:
                    for col_name in df.columns:
                        formatted_row[f"{row_name} {col_name}"] = df.loc[row_name, col_name]

                # Convert to DataFrame and clean columns
                temp_df = pd.DataFrame([formatted_row])
                temp_df = temp_df.drop(temp_df.columns[1:45], axis=1)  # Remove columns 2 to 45
                temp_df = temp_df.iloc[:, :-88]  # Remove last 88 columns

                # Append to list
                data_rows.append(temp_df)

        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    else:
        print(f"File {file_name} not found!")

# Combine all processed rows into a final DataFrame
final_df = pd.concat(data_rows, ignore_index=True)

# Fill missing values with empty string
final_df = final_df.fillna("")

# Display the cleaned DataFrame
final_df


,Date,"Average Price ('Unnamed: 1_level_0', 'Rice')","Average Price ('Unnamed: 2_level_0', 'Wheat')","Average Price ('Unnamed: 3_level_0', 'Atta (Wheat)')","Average Price ('Unnamed: 4_level_0', 'Gram Dal')","Average Price ('Unnamed: 5_level_0', 'Tur/Arhar Dal')","Average Price ('Unnamed: 6_level_0', 'Urad Dal')","Average Price ('Unnamed: 7_level_0', 'Moong Dal')","Average Price ('Unnamed: 8_level_0', 'Masoor Dal')","Average Price ('Unnamed: 9_level_0', 'Sugar')",...,"Puducherry ('Unnamed: 13_level_0', 'Vanaspati (Packed)')","Puducherry ('Unnamed: 14_level_0', 'Soya Oil (Packed)')","Puducherry ('Unnamed: 15_level_0', 'Sunflower Oil (Packed)')","Puducherry ('Unnamed: 16_level_0', 'Palm Oil (Packed)')","Puducherry ('Unnamed: 17_level_0', 'Gur')","Puducherry ('Unnamed: 18_level_0', 'Tea Loose')","Puducherry ('Unnamed: 19_level_0', 'Salt Pack (Iodised)')","Puducherry ('Unnamed: 20_level_0', 'Potato')","Puducherry ('Unnamed: 21_level_0', 'Onion')","Puducherry ('Unnamed: 22_level_0', 'Tomato')"
0,01/2023,37.62,32.05,36.81,70.45,110.45,105.44,101.82,93.53,41.45,...,,,,,,,,,,
1,02/2023,38.44,33.34,38.05,72.88,111.6,107.13,104.07,93.89,41.86,...,,,,,,,,,,
2,03/2023,38.77,30.52,35.3,72.61,112.28,107.18,104.51,92.59,41.72,...,153,,130,105,48,343,27,24,14,19
3,04/2023,39.09,29.4,34.77,72.83,115.18,107.62,105.79,93.08,41.57,...,151,,121,102,49,387,22,20,23,10
4,05/2023,38.48,28.74,34.23,73.71,116.68,108.23,107.29,93.11,42.3,...,150,,119,104,45,387,23,24,20,8
5,06/2023,39.28,28.86,34.15,74.64,122.08,110.43,108.96,92.28,42.53,...,150,,119,104,46,387,23,25,23,8
6,07/2023,39.89,29.29,34.74,74.91,131.24,112.55,110.12,92.06,42.98,...,149,,113,94,49,387,23,25,22,14
7,08/2023,40.98,29.59,34.72,75.21,135.25,114.24,110.94,91.98,43.07,...,149,,113,94,49,387,23,26,29,117
8,09/2023,41.9,30,35.34,78.59,141.57,115.73,111.88,92.66,43.38,...,,,,,,,,,,
9,10/2023,42.39,30.29,35.89,80.99,151.54,117.85,114.61,93.52,43.35,...,140,,110,90,50,393,22,33,35,15


In [6]:
os.chdir('/content/drive/MyDrive/Data/Data/2024-retail')

In [ ]:
import os

# Folder containing the files
folder_path = "/content/drive/MyDrive/Data/Data/Jan2025"  # Change this to your actual folder path

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    old_file = os.path.join(folder_path, filename)

    # Ensure it's a file (not a directory)
    if os.path.isfile(old_file):
        # Get file name without extension
        name, _ = os.path.splitext(filename)

        # Create new file name with .html extension
        new_file = os.path.join(folder_path, f"{name}.html")

        # Rename the file
        os.rename(old_file, new_file)

print("File extensions changed to .html successfully!")


File extensions changed to .html successfully!


In [8]:
import pandas as pd
import os

# Initialize an empty list to store rows
data_rows = []

# Loop through each file from 1.html to 31.html
for day in range(1, 12):
    file_name = f"1-{day}-24.html"

    if os.path.exists(file_name):
        try:
            # Read all tables in the HTML file
            df_list = pd.read_html(file_name)

            if df_list:
                df = df_list[0]  # Select the first table

                # Extract row names as index
                df = df.set_index(df.columns[0])  # First column is row names

                # Create a dictionary for the row
                formatted_row = {"Date": f"{day:02d}/2024"}

                for row_name in df.index:
                    for col_name in df.columns:
                        formatted_row[f"{row_name} {col_name}"] = df.loc[row_name, col_name]

                # Convert to DataFrame and clean columns
                temp_df = pd.DataFrame([formatted_row])
                temp_df = temp_df.drop(temp_df.columns[1:45], axis=1)  # Remove columns 2 to 45
                temp_df = temp_df.iloc[:, :-88]  # Remove last 88 columns

                # Append to list
                data_rows.append(temp_df)

        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    else:
        print(f"File {file_name} not found!")

# Combine all processed rows into a final DataFrame
final_df1 = pd.concat(data_rows, ignore_index=True)

# Fill missing values with empty string
final_df1 = final_df1.fillna("")

# Display the cleaned DataFrame
final_df1


,Date,"Average Price ('Unnamed: 1_level_0', 'Rice')","Average Price ('Unnamed: 2_level_0', 'Wheat')","Average Price ('Unnamed: 3_level_0', 'Atta (Wheat)')","Average Price ('Unnamed: 4_level_0', 'Gram Dal')","Average Price ('Unnamed: 5_level_0', 'Tur/Arhar Dal')","Average Price ('Unnamed: 6_level_0', 'Urad Dal')","Average Price ('Unnamed: 7_level_0', 'Moong Dal')","Average Price ('Unnamed: 8_level_0', 'Masoor Dal')","Average Price ('Unnamed: 9_level_0', 'Sugar')",...,"Ladakh ('Unnamed: 13_level_0', 'Vanaspati (Packed)')","Ladakh ('Unnamed: 14_level_0', 'Soya Oil (Packed)')","Ladakh ('Unnamed: 15_level_0', 'Sunflower Oil (Packed)')","Ladakh ('Unnamed: 16_level_0', 'Palm Oil (Packed)')","Ladakh ('Unnamed: 17_level_0', 'Gur')","Ladakh ('Unnamed: 18_level_0', 'Tea Loose')","Ladakh ('Unnamed: 19_level_0', 'Salt Pack (Iodised)')","Ladakh ('Unnamed: 20_level_0', 'Potato')","Ladakh ('Unnamed: 21_level_0', 'Onion')","Ladakh ('Unnamed: 22_level_0', 'Tomato')"
0,01/2024,43.03,30.99,36.19,82.66,152.34,121.98,115.27,93.44,44.29,...,,,,,,,,,,
1,02/2024,43.88,31.17,36.12,82.68,148.75,122.77,116.15,93.48,44.53,...,,,,,,,,,,
2,03/2024,44.01,30.9,36.13,83.48,148.76,123.38,116.64,93.45,44.38,...,,,,,,,,,,
3,04/2024,44.49,30.65,36.02,84.16,149.23,124.18,117.75,93.74,44.44,...,,,,,,,,,,
4,05/2024,43.92,30.59,35.8,83.78,151.84,123.91,116.84,92.9,44.49,...,,,,,,,,,,
5,06/2024,44.87,30.83,36.09,86.04,158.73,126.62,118.54,94.08,44.68,...,,,,,,,,,,
6,07/2024,44.95,31.17,36.2,88.62,162.63,126.87,118.79,94.15,44.92,...,,,,,,,,,,
7,08/2024,43.88,30.97,35.78,88.54,167.58,126.53,117.45,91.59,44.83,...,,,,,,,,,,
8,09/2024,43.23,30.75,35.8,91.3,163.83,124.14,114.54,89.09,44.42,...,,,,,,,,,,
9,10/2024,43.36,31.03,35.98,93.65,163.41,124.61,115.87,89.56,44.78,...,,,,,,,,,,


In [9]:
# Combine final_df and final_df1, keeping final_df first
new_df = pd.concat([final_df, final_df1], ignore_index=True)

# Display the combined DataFrame
new_df


,Date,"Average Price ('Unnamed: 1_level_0', 'Rice')","Average Price ('Unnamed: 2_level_0', 'Wheat')","Average Price ('Unnamed: 3_level_0', 'Atta (Wheat)')","Average Price ('Unnamed: 4_level_0', 'Gram Dal')","Average Price ('Unnamed: 5_level_0', 'Tur/Arhar Dal')","Average Price ('Unnamed: 6_level_0', 'Urad Dal')","Average Price ('Unnamed: 7_level_0', 'Moong Dal')","Average Price ('Unnamed: 8_level_0', 'Masoor Dal')","Average Price ('Unnamed: 9_level_0', 'Sugar')",...,"Ladakh ('Unnamed: 13_level_0', 'Vanaspati (Packed)')","Ladakh ('Unnamed: 14_level_0', 'Soya Oil (Packed)')","Ladakh ('Unnamed: 15_level_0', 'Sunflower Oil (Packed)')","Ladakh ('Unnamed: 16_level_0', 'Palm Oil (Packed)')","Ladakh ('Unnamed: 17_level_0', 'Gur')","Ladakh ('Unnamed: 18_level_0', 'Tea Loose')","Ladakh ('Unnamed: 19_level_0', 'Salt Pack (Iodised)')","Ladakh ('Unnamed: 20_level_0', 'Potato')","Ladakh ('Unnamed: 21_level_0', 'Onion')","Ladakh ('Unnamed: 22_level_0', 'Tomato')"
0,01/2023,37.62,32.05,36.81,70.45,110.45,105.44,101.82,93.53,41.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02/2023,38.44,33.34,38.05,72.88,111.6,107.13,104.07,93.89,41.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03/2023,38.77,30.52,35.3,72.61,112.28,107.18,104.51,92.59,41.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04/2023,39.09,29.4,34.77,72.83,115.18,107.62,105.79,93.08,41.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05/2023,38.48,28.74,34.23,73.71,116.68,108.23,107.29,93.11,42.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,06/2023,39.28,28.86,34.15,74.64,122.08,110.43,108.96,92.28,42.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,07/2023,39.89,29.29,34.74,74.91,131.24,112.55,110.12,92.06,42.98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,08/2023,40.98,29.59,34.72,75.21,135.25,114.24,110.94,91.98,43.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,09/2023,41.9,30,35.34,78.59,141.57,115.73,111.88,92.66,43.38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10/2023,42.39,30.29,35.89,80.99,151.54,117.85,114.61,93.52,43.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# prompt: final_df to csv

# Assuming final_df is already defined as in your provided code.
new_df.to_csv('final_df.csv', index=False)  # Save to current working directory
